In [1]:
%cd ..

/home/bhkuser/bhklab/katy/readii_2_roqc


In [30]:
from damply import dirs
import pandas as pd
from readii.process.label import getPatientIdentifierLabel
from readii.process.subset import getPatientIntersectionDataframes, getOnlyPyradiomicsFeatures
from readii_2_roqc.analysis.predict import load_signature_config
from readii.io.loaders import loadFileToDataFrame

import numpy as np
from pathlib import Path

In [31]:
clinical_file_path = dirs.RAWDATA / "TCIA_RADCURE/clinical/RADCURE_Clinical_v04_20241219.xlsx"

index_file_path = dirs.PROCDATA / "TCIA_RADCURE" / "features" / "pyradiomics" / "pyradiomics_RADCURE_index.csv"

feature_file_path = dirs.RESULTS / "TCIA_RADCURE" / "features" / "pyradiomics" / "pyradiomics_h4h_all_images_features" / "original_full_features.csv"

In [32]:
clinical_data = loadFileToDataFrame(clinical_file_path)
index_data = pd.read_csv(index_file_path)
feature_data = pd.read_csv(feature_file_path)

In [33]:
existing_pat_id = getPatientIdentifierLabel(clinical_data)

id_map = index_data["SampleID"]
id_map.index = [value[0:-5] for value in index_data["SampleID"]]
id_map = id_map.drop_duplicates()

clinical_data["SampleID"] = clinical_data[existing_pat_id].map(id_map)
clinical_data["Status_bin"] = clinical_data["Status"].apply(lambda x: 1 if x == "Dead" else 0)

In [34]:
opc_clinical_data = clinical_data[clinical_data["Ds Site"] == "Oropharynx"]

opc_hpv_clinical_data = opc_clinical_data.dropna(axis=0, subset=["SampleID", "HPV"])
opc_hpv_clinical_data = opc_hpv_clinical_data.set_index("SampleID", drop=True)


feature_data = feature_data.set_index("SampleID", drop=True)
feature_data = getOnlyPyradiomicsFeatures(feature_data)

In [35]:
clinical_subset, feature_subset = getPatientIntersectionDataframes(opc_hpv_clinical_data, feature_data, False, False)

In [36]:
outcome_series = clinical_subset['HPV'].apply(lambda x: 1 if x == "Yes, positive" else 0)

# Save out subset data

In [37]:
feature_subset.insert(0, "HPV_bin", outcome_series)

In [38]:
labelled_output_dir = Path(dirs.PROCDATA / "TCIA_RADCURE" / "features" / "labelled" / "HPV_pyradiomics")
labelled_output_dir.mkdir(parents=True, exist_ok=True)
feature_subset.to_csv( labelled_output_dir / "original_full_features.csv", index=True)

In [39]:
clinical_subset.to_csv(dirs.PROCDATA / "TCIA_RADCURE" / "clinical" / "RADCURE_Clinical_OPC_HPV_20250820.csv", index=True)

In [41]:
clinical_subset

,patient_id,Age,Sex,ECOG PS,Smoking PY,Smoking Status,Ds Site,Subsite,T,N,...,Date Local,Regional,Date Regional,Distant,Date Distant,2nd Ca,Date 2nd Ca,RADCURE-challenge,ContrastEnhanced,Status_bin
SampleID,,,,,,,,,,,,,,,,,,,,,
RADCURE-0005_0000,RADCURE-0005,62.6,Female,ECOG 0,50,Ex-smoker,Oropharynx,post wall,T4b,N2c,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0,0,1
RADCURE-0007_0002,RADCURE-0007,49.9,Male,ECOG 1,15,Ex-smoker,Oropharynx,Tonsil,T3,N2b,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0,1,0
RADCURE-0010_0004,RADCURE-0010,59.7,Female,ECOG 0,0,Non-smoker,Oropharynx,Tonsillar Fossa,T4b,N0,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0,0,0
RADCURE-0017_0010,RADCURE-0017,59.7,Male,ECOG 0,40,Current,Oropharynx,Soft Palate,T3,N0,...,NaT,NaN,NaT,NaN,NaT,Y,2003-06-15,0,0,0
RADCURE-0021_0014,RADCURE-0021,69.3,Male,ECOG 0,0,Non-smoker,Oropharynx,Tonsil,T4a,N2b,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RADCURE-4113_3321,RADCURE-4113,45.9,Male,ECOG 2,15,Current,Oropharynx,Tonsil,T4a,N2b,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,test,1,0
RADCURE-4124_3330,RADCURE-4124,68.3,Male,ECOG 1,0,Non-smoker,Oropharynx,Base of Tongue,T2,N2b,...,NaT,NaN,NaT,NaN,NaT,Y,2002-09-02,training,0,0
RADCURE-4126_3332,RADCURE-4126,58.3,Male,ECOG 0,50,Ex-smoker,Oropharynx,Tonsil Pillar,T2,N0,...,NaT,NaN,NaT,NaN,NaT,Y,2006-11-21,training,0,1


In [ ]:
clinical_subset.index.str.split(r'_[0-9]+', regex=True)

Index([['RADCURE-0005', '0000'], ['RADCURE-0007', '0002'],
       ['RADCURE-0010', '0004'], ['RADCURE-0017', '0010'],
       ['RADCURE-0021', '0014'], ['RADCURE-0025', '0018'],
       ['RADCURE-0031', '0023'], ['RADCURE-0035', '0025'],
       ['RADCURE-0037', '0027'], ['RADCURE-0043', '0032'],
       ...
       ['RADCURE-4100', '3313'], ['RADCURE-4102', '3314'],
       ['RADCURE-4105', '3316'], ['RADCURE-4107', '3318'],
       ['RADCURE-4108', '3319'], ['RADCURE-4113', '3321'],
       ['RADCURE-4124', '3330'], ['RADCURE-4126', '3332'],
       ['RADCURE-4127', '3333'], ['RADCURE-4129', '3335']],
      dtype='object', name='SampleID', length=418)

# Choi et al. signature

In [10]:
signature = load_signature_config(dirs.CONFIG / "signatures/choi_opc_hpv_2020.yaml")

In [32]:
signature

original_shape_Flatness                              4.754
original_shape_SphericalDisproportion                5.012
wavelet-HLH_firstorder_Mean                          9.017
wavelet-HLH_firstorder_Uniformity                    3.606
wavelet-HLH_glcm_ClusterShade                        4.692
wavelet-LHH_glcm_Idm                                 3.296
wavelet-LHH_glcm_Imc1                                4.201
wavelet-LHL_glszm_SmallAreaHighGrayLevelEmphasis    11.378
wavelet-LLH_glcm_Imc2                                4.503
dtype: float64

In [11]:
signature_feature_data = feature_subset[signature.index]

In [ ]:
from sklearn.linear_model import LogisticRegression, SGD

model = LogisticRegression(solver='liblinear').fit(signature_feature_data, outcome_series)

In [30]:
model.score(signature_feature_data, outcome_series)

0.7655502392344498

In [31]:
model.predict(signature_feature_data)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,